In [17]:
import pandas as pd
# data = pd.read_csv('D:/ShipSupplies/DA/data/raw.csv')
data = pd.read_csv('../data/동원산업통합_분류_1_RE.csv', encoding = 'cp949')

In [18]:
# 결측치 확인
missing_values = data.isnull().sum()
print(missing_values)

청구서번호              0
No.                0
Subject           22
Machinery          0
Assembly           0
청구품목               0
Part No.1          0
Part No.2      17747
key1               0
key2               0
리드타임_음수제거          0
청구량              118
견적               283
견적수량             118
견적화폐             118
견적단가               0
발주번호               0
발주처              118
발주               118
발주수량             118
발주금액             118
D/T             1090
미입고 기간         21084
리드타임               0
창고입고               0
창고입고수량             0
Control No.     6126
입고창고               0
창고출고            1663
창고출고수량             0
출고선박            1663
출고운반선           1663
선박입고           15427
선박입고수량             0
완료 여부          15439
dtype: int64


In [19]:
# 컬럼 삭제
df = data.drop(['Part No.2', '미입고 기간', '선박입고','완료 여부'], axis=1)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21084 entries, 0 to 21083
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   청구서번호        21084 non-null  object 
 1   No.          21084 non-null  int64  
 2   Subject      21062 non-null  object 
 3   Machinery    21084 non-null  object 
 4   Assembly     21084 non-null  object 
 5   청구품목         21084 non-null  object 
 6   Part No.1    21084 non-null  object 
 7   key1         21084 non-null  object 
 8   key2         21084 non-null  object 
 9   리드타임_음수제거    21084 non-null  int64  
 10  청구량          20966 non-null  float64
 11  견적           20801 non-null  object 
 12  견적수량         20966 non-null  float64
 13  견적화폐         20966 non-null  object 
 14  견적단가         21084 non-null  object 
 15  발주번호         21084 non-null  object 
 16  발주처          20966 non-null  object 
 17  발주           20966 non-null  object 
 18  발주수량         20966 non-null  float64
 19  발주금액

In [21]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
df = df.apply(le.fit_transform)

X = df[['Machinery', 'Assembly' , "Part No.1","청구품목"]]
y = df["key2"]
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=40, shuffle=True, test_size=0.2)

In [22]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 학습 데이터와 테스트 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,shuffle=True, random_state=42)

# XGBoost 모델 생성
model = xgb.XGBClassifier(objective='multi:softmax', num_class=61)

# 탐색할 하이퍼파라미터 그리드 축소
param_grid = {
    'max_depth': [3, 4],
    'learning_rate': [0.1, 0.01],
    'n_estimators': [100, 200]
}

# GridSearchCV를 사용하여 하이퍼파라미터 탐색
grid_search = GridSearchCV(model, param_grid, scoring='f1_macro', cv=3, n_jobs=-1)
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 확인
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

# 최적의 하이퍼파라미터로 모델 재학습
best_model = xgb.XGBClassifier(objective='multi:softmax', num_class=61, **best_params)
best_model.fit(X_train, y_train)

Best Hyperparameters: {'learning_rate': 0.1, 'max_depth': 4, 'n_estimators': 200}


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=4, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=200, n_jobs=None, num_class=61,
              num_parallel_tree=None, objective='multi:softmax', ...)

In [23]:
# 예측 수행
y_pred = best_model.predict(X_test)

# 선택한 평가 지표 계산
f1 = f1_score(y_test, y_pred, average='macro')  # 예시로 F1-score 사용

# 결과 출력
print("F1-score:", f1)


F1-score: 0.812909827167555


In [ ]:
from sklearn.metrics import f1_score

# 모델 예측
y_pred = model.predict(X_test)

# F1-score 계산
f1 = f1_score(y_test, y_pred, average='weighted')
print("F1-score:", f1)

In [256]:
data = pd.read_csv('../data/동원산업통합_분류_1_RE.csv', encoding = 'cp949')
# 컬럼 삭제
df = data.drop(['Part No.2', '미입고 기간', '선박입고','완료 여부'], axis=1)

In [257]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21084 entries, 0 to 21083
Data columns (total 31 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   청구서번호        21084 non-null  object 
 1   No.          21084 non-null  int64  
 2   Subject      21062 non-null  object 
 3   Machinery    21084 non-null  object 
 4   Assembly     21084 non-null  object 
 5   청구품목         21084 non-null  object 
 6   Part No.1    21084 non-null  object 
 7   key1         21084 non-null  object 
 8   key2         21084 non-null  object 
 9   리드타임_음수제거    21084 non-null  int64  
 10  청구량          20966 non-null  float64
 11  견적           20801 non-null  object 
 12  견적수량         20966 non-null  float64
 13  견적화폐         20966 non-null  object 
 14  견적단가         21084 non-null  object 
 15  발주번호         21084 non-null  object 
 16  발주처          20966 non-null  object 
 17  발주           20966 non-null  object 
 18  발주수량         20966 non-null  float64
 19  발주금액

In [258]:
# 제일 좋은 모델
X = list(df['Machinery']+ ' ' +df['Assembly']+  ' ' + df['Part No.1'] + ' ' + df['청구품목'])

# Subject
from sklearn.preprocessing import StandardScaler

y = df['key2']

In [259]:
#X,
y

0          COOLER
1          COOLER
2          COOLER
3          COOLER
4          COOLER
           ...   
21079    CYLINDER
21080    CYLINDER
21081        BELT
21082    CYLINDER
21083    CYLINDER
Name: key2, Length: 21084, dtype: object

In [260]:
le = preprocessing.LabelEncoder()
y = y.values.ravel()  # y를 1차원 배열로 변환
y = le.fit_transform(y)

In [261]:
type(y)
# y.shape

numpy.ndarray

In [262]:
from gensim.models import Word2Vec

# 리스트를 단어 시퀀스로 변환
sentences = [text.split() for text in X]

# Word2Vec 모델 학습
model = Word2Vec(sentences, vector_size=30, window=5, min_count=1, workers=4)

# 리스트 임베딩 생성
embedded_X = [model.wv[text.split()].mean(axis=0) for text in X]

In [263]:
# embedded_X
lengths = [len(text) for text in embedded_X]
average_length = sum(lengths) / len(lengths)
max_length = max(lengths)

print("Average length:", average_length)
print("Max length:", max_length)

Average length: 30.0
Max length: 30


In [264]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(embedded_X, y, test_size=0.2, random_state=42)

In [265]:
from tensorflow.keras.utils import to_categorical
# y값 카테고리 화
y_train_encoded = to_categorical(y_train, num_classes=61)
y_test_encoded = to_categorical(y_test, num_classes=61)

In [266]:
import numpy as np

X_train = np.array(X_train)
X_test = np.array(X_test)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

# X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1, 1)
# X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1, 1)

# X_train = np.expand_dims(X_train, axis=2)  # (데이터 개수, 시퀀스 길이, 1)
# X_test = np.expand_dims(X_test, axis=2)  # (데이터 개수, 시퀀스 길이, 1)

In [267]:
X_train.shape, X_test.shape

((16867, 30, 1, 1), (4217, 30, 1, 1))

In [268]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, GlobalMaxPooling1D, Dense

# 모델 구성
# model = Sequential()
# model.add(Embedding(input_dim=30, output_dim=100, input_length=30))
# model.add(Conv1D(32, kernel_size=3, activation='relu'))
# model.add(GlobalMaxPooling1D())
# model.add(Dense(128, activation='relu'))
# model.add(Dense(61, activation='softmax'))
# model.add(Dense(4, activation='softmax'))


# model = Sequential()
# model.add(Conv1D(32, 3, activation='relu', input_shape=(30, 100)))
# model.add(GlobalMaxPooling1D())
# model.add(Dense(128, activation='relu'))
# model.add(Dense(61, activation='softmax'))

model = Sequential()
model.add(Embedding(input_dim=3000, output_dim=100, input_length=100))
model.add(Conv1D(32, 3, activation='relu'))
model.add(GlobalMaxPooling1D())
model.add(Dense(128, activation='relu'))
model.add(Dense(61, activation='softmax'))


# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 요약 정보 출력
model.summary()

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_17 (Embedding)    (None, 100, 100)          300000    
                                                                 
 conv1d_20 (Conv1D)          (None, 98, 32)            9632      
                                                                 
 global_max_pooling1d_20 (Gl  (None, 32)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_35 (Dense)            (None, 128)               4224      
                                                                 
 dense_36 (Dense)            (None, 61)                7869      
                                                                 
Total params: 321,725
Trainable params: 321,725
Non-trainable params: 0
_______________________________________________

In [269]:
# 모델 학습
# model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
# model.fit(X_train, y_train_encoded, epochs=10, batch_size=32, validation_data=(X_test, y_test_encoded))
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
import matplotlib.pyplot as plt

# 손실 값 추출
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# 에포크 수
epochs = range(1, len(train_loss) + 1)

# 학습 곡선 그리기
plt.plot(epochs, train_loss, 'b-', label='Training Loss')
plt.plot(epochs, val_loss, 'r-', label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

Epoch 1/10


ValueError: in user code:

    File "D:\ShipSupplies\DA\venv\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "D:\ShipSupplies\DA\venv\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "D:\ShipSupplies\DA\venv\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "D:\ShipSupplies\DA\venv\lib\site-packages\keras\engine\training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "D:\ShipSupplies\DA\venv\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "D:\ShipSupplies\DA\venv\lib\site-packages\keras\engine\input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueError: Input 0 of lay

In [51]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# 모델 구성
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(width, height, channels)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(61, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 요약 정보 출력
model.summary()

NameError: name 'width' is not defined